In [ ]:
% load_ext autoreload
% autoreload 2

In [ ]:
!pip install .

In [ ]:
import folium
import geopandas as gpd
import pandas as pd
import os
import openspoor
from openspoor.mapservices import SpoortakMapservices, PUICMapservices
from openspoor.transformers import TransformerCoordinatesToSpoor, TransformerGeocodeToCoordinates, TransformerSpoortakToCoordinates
from openspoor.visualisations.trackmap import TrackMap, PlottingDataFrame

# Demo 1 - Set up trackmap and add points to plot

Making a map consists of setting up a TrackMap object and adding the objects you wish to plot to it. These objects can be given in Pandas DataFrames, which are displayed on a map with added aerial photographs of the Dutch tracks and zoomed to the location of interest.
Optionally, these outputs can be saved as .html files which can then be shared or used in applications.
Within notebooks, TrackMap objects are displayed if their value is requested at the end of a cell.

In [ ]:
m=TrackMap()
# Add pretty markers with colors and custom Font Awesome icons at custom locations
m.add(PlottingDataFrame({'lat': [52.08, 52.093],
                         'lon': [5.119, 5.107],
                         'value': [1, 2],
                         "marker": ['train', "eye"]},
                        colors={'value': {(0, 1.5): 'blue', (1.5, 3.0): 'orange'}},
                        marker_column="marker"))

# Add clickable markers with arrows to indicate directions at locations
m.add(PlottingDataFrame({'lat': [52.0874117], 'lon': [5.1156098], "rotation": [245], 'location': ['ProRail Entrance']},
                        colors='purple',
                        popup=['location'],
                        rotation_column="rotation"))

# Plot clickable circle(s) on a map
m.add(PlottingDataFrame({'lat': [52.086420], 'lon': [5.113101], 'radius': [10], 'object': ['switch']},
                        popup=['object'],
                        markertype='circle',
                        radius_column="radius"))
m.show()

# Load and create data


First step is to load and to obtain the data we will use for the example notebook. This step will take some as it will download all needed data. We will use and plot the following data sets:
1. PUIC: ProRail Unieke Identificatie Code. This is a "spoortakkenmodel" conform the bid00023 regulations. It is obtained via mapservices.prorail.nl from the Geleidingssystemen 007
2. Spoortakken: These contain Spoortak_identificatie and geometry and therefore lokale kilometrering.

In [ ]:
# get PUIC gdf
puic_mapservices = PUICMapservices()
puic_gdf = puic_mapservices.load_data()

# get spoortakken gdf
spoortak_mapservices = SpoortakMapservices()
spoortak_gdf = spoortak_mapservices.load_data()

# Plot function

In [ ]:
def create_folium_plot(new_layer, puic, spoortak):
    # create a bounding-box for automatically center and zoom the map
    min_lon, min_lat, max_lon, max_lat = spoortak.total_bounds

    # Create interactive map
    map_pgo = folium.Map(
        min_lat=min_lat,
        min_lon=min_lon,
        max_lat=max_lat,
        max_lon=max_lon,
        max_bounds=True,
        tiles=None
    )

    # automatically center and zoom the map if it is a geopandas dataframe
    if 'geometry' in new_layer.columns:
        min_lon, min_lat, max_lon, max_lat = new_layer.total_bounds
        folium.FitBounds([(min_lat, min_lon), (max_lat, max_lon)]).add_to(map_pgo)
    else:
        print("pandas")
        folium.FitBounds([(min_lat, min_lon), (max_lat, max_lon)]).add_to(map_pgo)

    # layer: openstreetmap
    fg = folium.FeatureGroup(name=f"openstreetmap", show=True)
    folium.TileLayer('openstreetmap').add_to(fg)
    map_pgo.add_child(fg)

    # layer: puic
    fg = folium.FeatureGroup(name=f"puic", show=True)
    folium.Choropleth(
        geo_data=puic.to_json(),
        fill_color='blue',
        fill_opacity=1,
        line_width=5,
        line_color='blue',
        line_opacity=1
    ).add_to(fg)
    map_pgo.add_child(fg)

    # layer: spoortakken
    fg = folium.FeatureGroup(name=f"spoortakken", show=True)
    folium.Choropleth(
        geo_data=spoortak.to_json(),
        fill_color='black',
        fill_opacity=1,
        line_width=5,
        line_color='black',
        line_opacity=1
    ).add_to(fg)
    map_pgo.add_child(fg)

    # layer: X, Y coordinates
    fg = folium.FeatureGroup(name=f"X, Y Coordinates", show=True)
    for indice, row in new_layer.iterrows():
        html = f'''<B>{row.case_no}</B>'''
        if 'geometry' in new_layer.columns:
            location = [row.geometry.y, row.geometry.x]
        else:
            location = [row.x, row.y]

        folium.Marker(
            location=location,
            popup=html,
            icon=folium.map.Icon(color='blue')).add_to(fg)

    map_pgo.add_child(fg)

    # add layer control to the map
    folium.LayerControl().add_to(map_pgo)

    # show the map
    return map_pgo

# Demo 2 - Obtain a track information based on X, Y coordinates
This demo uses the X,Y coordinates you have to find the track information of that point. It finds "spoortak", "geocode", "kilometrering" and "lokale kilometrering". Five coordinate examples cases are defined:
1. Case 1: A coordinate near a track. 
2. Case 2: A coordinate inside a switch.
3. Case 3: A coordinate on a crossing
4. Case 4: A coordinate near a crossing
5. Case 5: A coordinate outside the buffer distance of the tracks of 1.2 meters.

In [ ]:
# create dataframe of all the cases
xy_case_df = pd.DataFrame({'case_no': ["case_1", "case_2", "case_3", "case_4", "case_5"],
                           'x': [146506.901, 146970.582, 146445.417, 146465.756, 146406.901],
                           'y': [430192.467, 430102.380, 430101.289, 430102.479, 430192.467]})

# transform to a geopandas dataframe
xy_case_gdf = gpd.GeoDataFrame(xy_case_df,
                               geometry=gpd.points_from_xy(xy_case_df['x'], xy_case_df['y']),
                               crs="EPSG:28992")

In [ ]:
# set the coordinate transformer
coordinates_transformer = TransformerCoordinatesToSpoor()
coordinates_transformer = coordinates_transformer.fit(puic_gdf, spoortak_gdf)

# perform the transformation for our example cases
xy_extended_case_gdf = coordinates_transformer.transform(xy_case_gdf)
print(xy_extended_case_gdf)

In [ ]:
# Folium needs all gdf's in crs = '4326'
puic_gdf_crs = puic_gdf.to_crs(epsg='4326')
spoortak_gdf_crs = spoortak_gdf.to_crs(epsg='4326')
xy_case_gdf_crs = xy_extended_case_gdf.to_crs(epsg='4326')

In [ ]:
m = TrackMap()
m.add(PlottingDataFrame(xy_case_gdf_crs,  popup=['case_no']))
m.show()

In [ ]:
# plot
create_folium_plot(xy_case_gdf_crs, puic_gdf_crs, spoortak_gdf_crs)

# Demo 3 - Obtain a X, Y coordinates based on track information

The second demo is the other way around. You already have a "spoortak" and its "lokale_kilometrering" and now we want to know what the X, Y Coordinate or GPS is. 

In [ ]:
# create dataframe of all the cases
spoortak_case_df = pd.DataFrame(
    {'case_no': ["case_1", "case_2"],
     'SPOOR_ID': ['152_4123V_30.7', '152_4123V_30.7'],
     'lokale_kilometrering': [2, 18]
     }
)

In [ ]:
# set the spoortak transformer for 'Rijksdriehoek'
spoortak_transformer = TransformerSpoortakToCoordinates(
    'SPOOR_ID',
    'lokale_kilometrering',
    coordinate_system='Rijksdriehoek'  # 'GPS' if you want the GPS coordinates
)
spoortak_transformer = spoortak_transformer.fit(spoortak_gdf)

# perform the transformation for our example cases
spoortak_case_extended_df = spoortak_transformer.transform(spoortak_case_df)

# transform to a geopandas dataframe
spoortak_case_extended_gdf = gpd.GeoDataFrame(spoortak_case_extended_df,
                                              geometry=gpd.points_from_xy(spoortak_case_extended_df['x'],
                                                                          spoortak_case_extended_df['y']),
                                              crs="epsg:28992")
spoortak_case_extended_gdf

In [ ]:
# Folium needs all gdf's in crs = '4326'
spoortak_case_extended_gdf_crs = spoortak_case_extended_gdf.to_crs(epsg='4326')

In [ ]:
m = TrackMap()
m.add(PlottingDataFrame(spoortak_case_extended_gdf_crs, popup=['case_no']))
m.show()

In [ ]:
# plot
create_folium_plot(spoortak_case_extended_gdf_crs, puic_gdf_crs, spoortak_gdf_crs)

# Demo 4 -  Obtain a GPS coordinates based on track information

This time you already have a "geocode" and its "lokale_kilometrering" and we want to know what the X, Y Coordinate or GPS is. 
Two simple cases on the same "spoortak". You can see in the plot that X, Y coordinate is of the "spoorhartlijn". This can be explaine by the fact that "geocode" and "geocode kilometrering" can have multiple "spoortakken". 

In [ ]:
# create dataframe of all the cases
geocode_case_df = pd.DataFrame(
    {
        'case_no': ['case_1', 'case_2'],
        'Geocode': ['112', '009'],
        'geocode_km': [77, 115.208]
    }
)

In [ ]:
# set the geocode transformer for 'GPS'
geocode_transformer = TransformerGeocodeToCoordinates(
    geocode_column='Geocode',
    geocode_km_column='geocode_km',
    coordinate_system='GPS'  # 'Rijksdriehoek' if you want the GPS coordinates
)

# perform the transformation for our example cases
geocode_case_extended_df = geocode_transformer.transform(geocode_case_df)

In [ ]:
m = TrackMap()
m.add(PlottingDataFrame(geocode_case_extended_df, lat_column='x', lon_column='y', popup=['case_no']))
m.show()

In [ ]:
# plot
create_folium_plot(geocode_case_extended_df, puic_gdf_crs, spoortak_gdf_crs)